In [1]:
import sys
import os
from collections import namedtuple
import pandas as pd
import numpy as np
Item = namedtuple("Item", ['index', 'value', 'weight'])

In [2]:
current_dir = os.getcwd()
data1_dir = './data/ks_4_0'

## Step1: Read files


In [39]:
with open(data1_dir, 'r') as input_data_file:
    input_data = input_data_file.read()
    
print(input_data)


4 11
8 4
10 5
15 8
4 3



## Step2: Preprocess the data

In [40]:
# Initialise the variables
# parse the input
lines = input_data.split('\n')

firstLine = lines[0].split()
item_count = int(firstLine[0])
capacity = int(firstLine[1])

items = []

# repeat for all

for i in range(1, item_count+1):
    line = lines[i]
    parts = line.split()
    items.append(Item(i-1, int(parts[0]), int(parts[1])))

In [41]:
items

[Item(index=0, value=8, weight=4),
 Item(index=1, value=10, weight=5),
 Item(index=2, value=15, weight=8),
 Item(index=3, value=4, weight=3)]

## Step3: Solver implementation

In [42]:
# Algo 1: take every item until it becomes full
value = 0
weight = 0
taken = [0]*len(items)

for item in items:
    if weight + item.weight <= capacity:
        taken[item.index] = 1
        value += item.value
        weight += item.weight

In [43]:
items

[Item(index=0, value=8, weight=4),
 Item(index=1, value=10, weight=5),
 Item(index=2, value=15, weight=8),
 Item(index=3, value=4, weight=3)]

## Step4: post-processing output

In [45]:
output_data = str(value) + ' ' + str(0) + '\n'
output_data += ' '.join(map(str, taken))
output_data

'18 0\n1 1 0 0'

# Improvise to df version

## Step1: Read files


In [3]:
with open(data1_dir, 'r') as input_data_file:
    input_data = input_data_file.read()
    
print(input_data)


4 11
8 4
10 5
15 8
4 3



## Step2: Preprocess the data

In [4]:
lines = input_data.split('\n')
lines

# get number of item available & max_capa
item_count = int(lines[0].split()[0])
max_capa = int(lines[0].split()[1])

for i in range(1, item_count+1):
    line = lines[i]
    parts = line.split()
    if i == 1: # initialise new variables
        values = [int(parts[0])]
        weight = [int(parts[1])]
    else:
        values.append(int(parts[0]))
        weight.append(int(parts[1]))

available_item_df = pd.DataFrame({"values":values, "weight":weight})
available_item_df

,values,weight
0,8,4
1,10,5
2,15,8
3,4,3


## Step3: Solver implementation

In [55]:
# Algo 1: take every item until it becomes full [improvised to df version]
value = 0
weight = 0
taken = [0]*len(available_item_df)
taken

for index, row in available_item_df.iterrows():
    if weight + available_item_df['weight'][index] <= max_capa:
        taken[index] = 1
        value += available_item_df['values'][index]
        weight += available_item_df['weight'][index]

In [59]:
# Algo 2: sort by highest value first, take until full [improvised to df version]
value = 0
weight = 0
taken = [0]*len(available_item_df)
taken

available_item_df.sort_values("values", ascending=False, inplace=True)

for index, row in available_item_df.iterrows():
    if weight + available_item_df['weight'][index] <= max_capa:
        taken[index] = 1
        value += available_item_df['values'][index]
        weight += available_item_df['weight'][index]

In [ ]:
# Algo 3: sort by highest value-weight ratio first, take until full [improvised to df version]
value = 0
weight = 0
taken = [0]*len(available_item_df)


available_item_df["value_weight_ratio"] = available_item_df['values']/available_item_df['weight']
available_item_df.sort_values("weight", ascending=True, inplace=True)
available_item_df.sort_values("value_weight_ratio", ascending=False, inplace=True)

for index, row in available_item_df.iterrows():
    if weight + available_item_df['weight'][index] <= max_capa:
        taken[index] = 1
        value += available_item_df['values'][index]
        weight += available_item_df['weight'][index]

In [13]:
# Algo 4: dynamic programming - bottom-up approach (filling up the table)

value_list = available_item_df['values'].to_list()
weight_list = available_item_df['weight'].to_list()
# value_list = [16,19,23,28]
# weight_list =[2,3,4,5]
number_of_item = len(weight_list)
value = 0
weight = 0
taken = [0]*len(available_item_df)
# max_capa_1 =7

dp_tbl = pd.DataFrame(np.zeros((number_of_item+1, max_capa+1)))
dp_picked = pd.DataFrame(np.zeros((number_of_item+1, max_capa+1)))

for i in range(1, number_of_item+1):
    for w in range(0, max_capa+1):

        if weight_list[i-1] <= w:
            dp_tbl.iloc[i, w] = max(dp_tbl.iloc[i-1, w], value_list[i-1] + dp_tbl.iloc[i-1, (w - weight_list[i-1])] )
            dp_picked.iloc[i, w] = 1
        else:
            dp_tbl.iloc[i, w] = dp_tbl.iloc[i-1, w] 
            dp_picked.iloc[i, w] = 0


w = max_capa
taken_index =list()
for i in range(number_of_item,0,-1):
    if dp_picked.iloc[i, w] == True:
        taken_index.append(i-1)
        w = w - weight_list[i-1]

for n in taken_index:
    value += available_item_df['values'][n]
    weight += available_item_df['weight'][n]
    taken[n] = 1





## Step4: Post-processsing

In [60]:
output_data = str(value) + ' ' + str(0) + '\n'
output_data += ' '.join(map(str, taken))
output_data

'19 0\n0 0 1 1'

In [61]:
max_capa

11

In [62]:
available_item_df

,values,weight
2,15,8
1,10,5
0,8,4
3,4,3
